In [16]:
from math import sqrt, log
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np
import sys
ROOT_DIR = "../"
sys.path.append(ROOT_DIR)
from prochain_transformer.modules.extra_layers import UniformAttentionMask

mask = [False,False,False,False]
mask_layer = UniformAttentionMask()

In [17]:
BATCH_SIZE = 1
seq_len = 4          # sequence length (here we assume the same for input/output, but it can be different)
heads = 1            # number of heads (multi-head attention)
d_model = 12         # Embedding dimension (?)
d_query_key = 9      # Dimension for the query and key vectors

# x = torch.ones(BATCH_SIZE,seq_len,d_model) # d_model 
x = torch.rand(BATCH_SIZE,seq_len,d_model) # d_model 
multi_projection = nn.Linear(d_model, d_query_key * heads, bias=False)


query_projection = nn.Linear(d_model, d_query_key * heads)
key_projection = nn.Linear(d_model, d_query_key * heads)

query = query_projection(x)
key = key_projection(x)

query_view = query.view((BATCH_SIZE, seq_len, heads, -1))
key_view = key.view((BATCH_SIZE, seq_len, heads, -1))

scale = 1.0 / sqrt(d_query_key)

scores = scale *torch.einsum("blhe,bshe->bhls", query_view, key_view)
scores_mask = mask_layer(scores,mask)
print(scores_mask)


tensor([[[[-0.1199, -0.0072, -0.1170,  0.0055],
          [-0.0676, -0.0626, -0.1043, -0.0312],
          [-0.0723,  0.0187, -0.0694,  0.0611],
          [-0.1176, -0.1026, -0.1503, -0.0732]]]],
       grad_fn=<MaskedFillBackward0>)


In [18]:
A_mask = torch.softmax(scores_mask, dim=-1)
A = torch.softmax(scores, dim=-1)

value_projection = nn.Linear(d_model, d_model *heads)
values = value_projection(x)
values_view = values.view(BATCH_SIZE, seq_len, heads, -1)

V_mask = torch.einsum("bhls,bshd->blhd", A_mask, values_view)
V = torch.einsum("bhls,bshd->blhd", A, values_view)

print(A_mask)
print(A)
print(V_mask)
print(V)

tensor([[[[0.2350, 0.2630, 0.2356, 0.2664],
          [0.2496, 0.2509, 0.2406, 0.2589],
          [0.2358, 0.2583, 0.2365, 0.2694],
          [0.2482, 0.2520, 0.2402, 0.2595]]]], grad_fn=<SoftmaxBackward0>)
tensor([[[[0.2350, 0.2630, 0.2356, 0.2664],
          [0.2496, 0.2509, 0.2406, 0.2589],
          [0.2358, 0.2583, 0.2365, 0.2694],
          [0.2482, 0.2520, 0.2402, 0.2595]]]], grad_fn=<SoftmaxBackward0>)
tensor([[[[-0.7423,  0.5358,  0.2884, -0.1044, -0.4614, -0.0761,  0.2985,
            0.5252,  0.1182,  0.3812, -0.0836, -0.6239]],

         [[-0.7415,  0.5299,  0.2883, -0.1036, -0.4668, -0.0745,  0.3012,
            0.5246,  0.1197,  0.3803, -0.0820, -0.6250]],

         [[-0.7428,  0.5359,  0.2884, -0.1037, -0.4619, -0.0754,  0.2995,
            0.5250,  0.1186,  0.3811, -0.0827, -0.6244]],

         [[-0.7416,  0.5304,  0.2883, -0.1037, -0.4663, -0.0746,  0.3010,
            0.5247,  0.1195,  0.3804, -0.0821, -0.6249]]]],
       grad_fn=<ViewBackward0>)
tensor([[[[-0.7423,  

In [19]:
value_projection = nn.Linear(d_model, d_model *heads)
values = value_projection(x)
values_view = values.view(BATCH_SIZE, seq_len, heads, -1)

V_mask = torch.einsum("bhls,bshd->blhd", A, values_view)
V = torch.einsum("bhls,bshd->blhd", A, values_view)

print(V_mask.shape)
print(V.shape)
print(V_mask)
print(V)

torch.Size([1, 4, 1, 12])
torch.Size([1, 4, 1, 12])
tensor([[[[-0.0204,  0.2157,  0.1236,  0.5567,  0.9937, -0.0930, -0.7458,
           -0.3583, -0.3070, -0.1132, -0.1033, -0.2142]],

         [[-0.0183,  0.2218,  0.1233,  0.5560,  0.9940, -0.0936, -0.7424,
           -0.3679, -0.3067, -0.1060, -0.1059, -0.2192]],

         [[-0.0209,  0.2170,  0.1230,  0.5571,  0.9942, -0.0927, -0.7453,
           -0.3590, -0.3070, -0.1134, -0.1032, -0.2154]],

         [[-0.0185,  0.2212,  0.1233,  0.5561,  0.9940, -0.0935, -0.7427,
           -0.3670, -0.3067, -0.1067, -0.1057, -0.2188]]]],
       grad_fn=<ViewBackward0>)
tensor([[[[-0.0204,  0.2157,  0.1236,  0.5567,  0.9937, -0.0930, -0.7458,
           -0.3583, -0.3070, -0.1132, -0.1033, -0.2142]],

         [[-0.0183,  0.2218,  0.1233,  0.5560,  0.9940, -0.0936, -0.7424,
           -0.3679, -0.3067, -0.1060, -0.1059, -0.2192]],

         [[-0.0209,  0.2170,  0.1230,  0.5571,  0.9942, -0.0927, -0.7453,
           -0.3590, -0.3070, -0.1134, -0.10

In [20]:
out_projection = nn.Linear(d_model*heads, d_model)

In [21]:
V_mask = V_mask.reshape(BATCH_SIZE,seq_len,-1)
V = V.reshape(BATCH_SIZE,seq_len,-1)
print(V_mask.shape)

torch.Size([1, 4, 12])


In [22]:
out_projection = nn.Linear(d_model*heads, d_model)

In [23]:
out = out_projection(V)
out_mask = out_projection(V_mask)
print(out_mask.shape)
print(abs(out)>abs(out_mask))
print(out)
print(out_mask)

torch.Size([1, 4, 12])
tensor([[[False, False, False, False, False, False, False, False, False, False,
          False, False],
         [False, False, False, False, False, False, False, False, False, False,
          False, False],
         [False, False, False, False, False, False, False, False, False, False,
          False, False],
         [False, False, False, False, False, False, False, False, False, False,
          False, False]]])
tensor([[[-0.3159,  0.2257, -0.0707, -0.3978,  0.1626, -0.2249,  0.3202,
           0.4620, -0.0756,  0.2144,  0.1905,  0.1502],
         [-0.3179,  0.2219, -0.0688, -0.3994,  0.1629, -0.2202,  0.3192,
           0.4622, -0.0770,  0.2129,  0.1924,  0.1496],
         [-0.3156,  0.2252, -0.0706, -0.3984,  0.1632, -0.2249,  0.3198,
           0.4625, -0.0756,  0.2140,  0.1904,  0.1502],
         [-0.3177,  0.2223, -0.0690, -0.3993,  0.1629, -0.2207,  0.3193,
           0.4622, -0.0769,  0.2130,  0.1922,  0.1497]]],
       grad_fn=<ViewBackward0>)
tenso